In [1]:
# This code here is intended to create a custom keras layer.
# This layer takes as input the path to the image
# It outputs an array that can be given as an input to a convolutional neural network

In [1]:
#Create constants and import libraries
UNPROCESSED_DIRECTORY = "CNN_synth_testset"
PROCESSED_DIRECTORY = "preprocessedImages"
X_DIM = 256
Y_DIM = 256
import os
import numpy as np
from PIL import Image as imLib
from keras.layers import Layer
from keras import models
from keras import layers
from keras.utils import Sequence
from math import ceil
import time


In [2]:
#Build custom layer

class ImageBatchSequence(Sequence):
    def __init__(self, x_set, batch_size):
        self.x = x_set
        y_set = [(1 if "FAKE" in x else 0) for x in x_set] #This makes the output to be 1 (AI-generated) if FAKE is in pathname.
        #                                                  # Otherwise, it makes the output 0 (not AI-generated)
        self.y = y_set
        self.batch_size = batch_size
    def __len__(self):
        return ceil(len(self.x)/self.batch_size)
    def __getitem__(self, index):
        low = index * self.batch_size
        high = min(low+self.batch_size, len(self.x))
        batch_x = self.x[low : high]
        batch_y = self.y[low : high]
        image_x_batch = []
        for imPath in batch_x:
            image = np.asarray(imLib.open(imPath)).reshape(256,256,3)
            time.sleep(.5)
            image_x_batch.append(image)
        return np.array(image_x_batch), np.array(batch_y)


In [6]:
# This is just to verify that the Keras Generator works.
# You should comment this out before importing.
# However, I think we should reorder all those files / structure later.
# In other words, we should make a better heirarchy for our code.
model = models.Sequential()
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
tempList=['./CNN_synth_testset\\biggan\\0_real\\0--n01440764_5043.png', 
          './CNN_synth_testset\\biggan\\0_real\\0--n01440764_8600.png', 
          './CNN_synth_testset\\biggan\\0_real\\10--n01530575_10395.png',
          './CNN_synth_testset\\biggan\\0_real\\100--n01860187_3168.png', 
          './CNN_synth_testset\\biggan\\0_real\\101--n01871265_1128.png']
ImageBatchGenerator = ImageBatchSequence(x_set = tempList, batch_size = 2)
model.add( layers.Conv2D( 1, (8,8), activation='relu', input_shape=(256,256,3,)  ) )
model.add( layers.Flatten() )
model.add(layers.Dense(1, activation='relu'))
model.summary()
history = model.fit(ImageBatchGenerator, epochs = 2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 249, 249, 1)       193       
                                                                 
 flatten_2 (Flatten)         (None, 62001)             0         
                                                                 
 dense_2 (Dense)             (None, 1)                 62002     
                                                                 
Total params: 62,195
Trainable params: 62,195
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
3/3 [==============================] - 3s 757ms/step - loss: nan - accuracy: 1.0000
Epoch 2/2
3/3 [==============================] - 3s 762ms/step - loss: nan - accuracy: 1.0000


In [4]:
#Check python version and whetehr GPU is there
!python --version
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_gpu_available()


Python 3.10.13
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11950116431221218201
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3643801600
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1686283512049067983
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
2.10.0
Num GPUs Available:  1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True